# Flux Sweeping via SRS

[srsinst pacakge](https://github.com/thinkSRS/srsinst.dc205)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
from flux_fit import *
from utils import *

Connect to the instruments

In [ ]:
!python -m srsinst.dc205

In [ ]:
client = Labber.connectToServer(timeout=None)
instrument_list = client.getListOfInstruments()
instruments = create_instrument_connections(client, instrument_list)

In [ ]:
vs = connect_SRS()
VNA = instruments['Agilent Network Analyzer E5071B']
DA = instruments['Vaunix Lab Brick Digital Attenuator']
SMU = instruments['Keithley 2400 SourceMeter']
LO= instruments['SignalCore SC5511A Signal Generator_10002F25']
Drive= instruments['SignalCore SC5511A Signal Generator_1000334C']
TWPA_PUMP = instruments['Rohde&Schwarz RF Source']

initialize_instruments(VNA, DA, SMU, LO, Drive, vs, TWPA_PUMP)

Since we only need the VNA and SRS for this measurement- lets make sure they are connected and ready to go. 

In [ ]:
vs.check_id()
VNA.isRunning()

Setting up the paths for storing the Labber file.

In [ ]:
# get the time module
import time
import os
import logging

SPATH = r"G:\Shared drives\Labber\Data\{}\{}\Data_{}\\".format(time.strftime("%Y"), time.strftime("%m"), time.strftime("%m%d"))

figpath = SPATH + r"Figures\\"
if not os.path.exists(figpath):
    os.makedirs(figpath)
print(f"Saving path of Labber data to {SPATH}")
print(f"Saving figures to {figpath}")
logging.basicConfig(filename=SPATH+f'MEASUREMENTLOG_{time.strftime("%Y%m%d_%H%M%S")}.log',filemode='w',level=logging.INFO)


Either set the VNA settings manually and grab them.

In [ ]:
VNA.getInstrConfig()

Or just set them in the code. 

```python

In [ ]:
VNA.setValue('Output enabled',True)
VNA.setValue('Range type','Start - Stop')
VNA.setValue('Start frequency', 5.02E9)
VNA.setValue('Stop frequency', 5.09E9)
VNA.setValue('# of averages', 20)
VNA.setValue('Average',True)
VNA.setValue('Wait for new trace', True)

Testing the VNA settings by grabbing a trace

In [ ]:
dBG = VNA.getValue('S21')
xBG = np.arange(dBG['t0'],dBG['t0']+dBG['shape'][0]*dBG['dt'],dBG['dt'])
zBG = dBG['y']
plt.plot(zBG)
plt.show()

In [ ]:
voltages = np.arange(-74E-3,72E-3,2E-3)
# reverse order
#voltages = np.flip(voltages)
voltages, len(voltages)````

(array([-7.4000000e-02, -7.2000000e-02, -7.0000000e-02, -6.8000000e-02,
        -6.6000000e-02, -6.4000000e-02, -6.2000000e-02, -6.0000000e-02,
        -5.8000000e-02, -5.6000000e-02, -5.4000000e-02, -5.2000000e-02,
        -5.0000000e-02, -4.8000000e-02, -4.6000000e-02, -4.4000000e-02,
        -4.2000000e-02, -4.0000000e-02, -3.8000000e-02, -3.6000000e-02,
        -3.4000000e-02, -3.2000000e-02, -3.0000000e-02, -2.8000000e-02,
        -2.6000000e-02, -2.4000000e-02, -2.2000000e-02, -2.0000000e-02,
        -1.8000000e-02, -1.6000000e-02, -1.4000000e-02, -1.2000000e-02,
        -1.0000000e-02, -8.0000000e-03, -6.0000000e-03, -4.0000000e-03,
        -2.0000000e-03,  6.9388939e-17,  2.0000000e-03,  4.0000000e-03,
         6.0000000e-03,  8.0000000e-03,  1.0000000e-02,  1.2000000e-02,
         1.4000000e-02,  1.6000000e-02,  1.8000000e-02,  2.0000000e-02,
         2.2000000e-02,  2.4000000e-02,  2.6000000e-02,  2.8000000e-02,
         3.0000000e-02,  3.2000000e-02,  3.4000000e-02,  3.60000

In [ ]:
labber_file_name = "NBRL1A_25_mK"

lfVNA = Labber.createLogFile_ForData(os.path.join(SPATH, f'{labber_file_name}_flux_sweep'),
                                      [{'name':'VNA - S21','complex':True,'vector':True,'x_name':'Frequency','x_unit':'Hz'}],
                                      step_channels=[{'name':'Voltage','values':voltages,'unit':'V'}])

In [ ]:
for voltage in tqdm(voltages, desc='Voltage Sweep', unit='V'):  # Add tqdm progress bar
    print(voltage)
    logging.info(f'Starting Voltage = {voltage:.3f}\\n')
    set_flux_bias_srs(voltage, step=5E-5)
    sleep(0.05)
    dData = VNA.getValue('S21')
    zData = dData['y']
    xBG = np.arange(dData['t0'],dData['t0']+dData['shape'][0]*dData['dt'],dData['dt'])
    td2 = Labber.getTraceDict(zData,x0=xBG[0],x1=xBG[-1])
    lfVNA.addEntry({'VNA - S21':td2})

Ensure that the flux bias is set back to zero.

In [ ]:
set_flux_bias_srs(0, step=5E-5)